In [69]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/hojjatk/mnist-dataset")

Skipping, found downloaded files in "./mnist-dataset" (use force=True to force download)


In [70]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary
import gzip

device =  "cuda" if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
def read_idx_images(filename):
    with open(filename, 'rb') as f:
        data = f.read()
    magic = int.from_bytes(data[0:4], 'big')
    num_images = int.from_bytes(data[4:8], 'big')
    rows = int.from_bytes(data[8:12], 'big')
    cols = int.from_bytes(data[12:16], 'big')
    images = np.frombuffer(data, dtype=np.uint8, offset=16)
    images = images.reshape(num_images, 1, rows, cols).astype(np.float32) / 255.0
    
    # MNIST standardization
    mean = 0.1307
    std = 0.3081
    images = (images - mean) / std
    
    return images

def read_idx_labels(filename):
    with open(filename, 'rb') as f:   # <-- normal binary open
        data = f.read()
        
    magic = int.from_bytes(data[0:4], 'big')
    num_labels = int.from_bytes(data[4:8], 'big')
    
    labels = np.frombuffer(data, dtype=np.uint8, offset=8)
    return labels


In [72]:
class DataSet(Dataset):
    def __init__(self, images , labels):
        self.images = torch.tensor(images, dtype = torch.float32)
        self.labels = torch.tensor(labels, dtype = torch.long)
        
    def __len__(self):
        return(len(self.images))
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [73]:
test_Ipath=r"/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/cnn/mnist-dataset/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte"
test_Lpath=r"/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/cnn/mnist-dataset/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte"
train_Ipath=r"/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/cnn/mnist-dataset/train-images-idx3-ubyte/train-images-idx3-ubyte"
train_Lpath=r="/home/namankarki/Naman/3_months_of_data-science/Deep_Learning_Pytorch/cnn/mnist-dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte"



test_images = read_idx_images(test_Ipath)
test_labels = read_idx_labels(test_Lpath)
train_images = read_idx_images(train_Ipath)
train_labels = read_idx_labels(train_Lpath)


In [74]:
train_dataset = DataSet(train_images, train_labels)
test_dataset = DataSet(test_images, test_labels)

In [75]:
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_dataloader = DataLoader(train_dataset, batch_size = 100, shuffle = False)

In [76]:
data=iter(train_dataloader)
image,label = next(data)
print(image.shape, label.shape)


torch.Size([64, 1, 28, 28]) torch.Size([64])


In [77]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32,3), #yesle dine output vanya of shape(N,32,26,26)
            nn.ReLU(),
            nn.Conv2d(32, 64, 3), #(N,64,24,24)
            nn.ReLU(),
            nn.MaxPool2d(2), #(N,64,12,12)
            
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),#(N,64*12*12)
            nn.Linear(64*12*12, 128),
            nn.ReLU(),
            nn.Linear(128,10)
        )
    
    def forward(self,x):
        x = self.features(x)
        x=self.classifier(x)
        return x

model = MyModel().to(device) 

In [78]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(params=model.parameters(), lr=0.001, betas=(0.9,0.999), eps=1e-8)

In [79]:
epochs = 20

for epoch in range(epochs):
    total_train_loss = 0
    total_train_acc = 0
    
    model.train()
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device).long()
        
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item() * labels.size(0)
        _, preds = torch.max(logits, 1)
        total_train_acc += (preds == labels).sum().item()
    
    avg_train_loss = total_train_loss / len(train_dataset)
    avg_train_acc = total_train_acc / len(train_dataset)
    
    # Validation
    val_loss = 0
    val_acc = 0
    model.eval()
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device).long()
            logits = model(images)
            loss = criterion(logits, labels)
            val_loss += loss.item() * labels.size(0)
            _, preds = torch.max(logits, 1)
            val_acc += (preds == labels).sum().item()
    
    avg_val_loss = val_loss / len(test_dataset)
    avg_val_acc = val_acc / len(test_dataset)
    
    print(f"Epoch {epoch+1}/{epochs}: "
          f"Train Loss = {avg_train_loss:.4f}, Train Acc = {avg_train_acc:.4f}, "
          f"Val Loss = {avg_val_loss:.4f}, Val Acc = {avg_val_acc:.4f}")


Epoch 1/20: Train Loss = 0.1159, Train Acc = 0.9642, Val Loss = 0.2096, Val Acc = 5.9371
Epoch 2/20: Train Loss = 0.0349, Train Acc = 0.9890, Val Loss = 0.1076, Val Acc = 5.9660
Epoch 3/20: Train Loss = 0.0214, Train Acc = 0.9929, Val Loss = 0.1108, Val Acc = 5.9643
Epoch 4/20: Train Loss = 0.0151, Train Acc = 0.9950, Val Loss = 0.0705, Val Acc = 5.9780
Epoch 5/20: Train Loss = 0.0108, Train Acc = 0.9966, Val Loss = 0.0461, Val Acc = 5.9857
Epoch 6/20: Train Loss = 0.0096, Train Acc = 0.9964, Val Loss = 0.0228, Val Acc = 5.9923
Epoch 7/20: Train Loss = 0.0075, Train Acc = 0.9977, Val Loss = 0.0737, Val Acc = 5.9755
Epoch 8/20: Train Loss = 0.0066, Train Acc = 0.9977, Val Loss = 0.0256, Val Acc = 5.9927
Epoch 9/20: Train Loss = 0.0046, Train Acc = 0.9987, Val Loss = 0.0285, Val Acc = 5.9910
Epoch 10/20: Train Loss = 0.0048, Train Acc = 0.9985, Val Loss = 0.0059, Val Acc = 5.9981
Epoch 11/20: Train Loss = 0.0052, Train Acc = 0.9982, Val Loss = 0.0224, Val Acc = 5.9926
Epoch 12/20: Train 